In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 24.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=45f816147aaad52700bed6566228096e54d19c9e80715a57917e5348389be031
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark=(SparkSession.builder.appName("...").getOrCreate())

In [4]:
sc=spark.sparkContext
sqlContext=spark

In [5]:
import pyspark.ml

In [6]:
from pyspark.ml.feature import Binarizer

In [7]:
from pyspark.sql.readwriter import DataFrameWriterV2
#Data Types
#Local vector
import numpy as np
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
#Use a Numpy array as a dense vector.
dv1=np.array([1.0,0.0,3.0])
dv1
#Use a Python list as a dense vector
dv2=[1.0,0.0,3.0]
DataFrameWriterV2
#Create a SparseVector.
sv1=Vectors.sparse(3,[0,2],[1.0,3.0])
sv1
#Use a single-column Scipy csc_matrix as a sparese vector.
sv2=sps.csc_matrix((np.array([1.0,3.0]), np.array([0,2]), np.array([0,2])),shape=(3,1))
sv2

<3x1 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Column format>

In [8]:
#Labled point
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

#Create a labeled point with a positive label and a dense feature  vector.
pos=LabeledPoint(1.0,[1.0,0.0,3.0])

#Create a labeled point with a negative label and a dense feature  vector.
neg=LabeledPoint(0.0,SparseVector(3,[0,2],[1.0,3.0]))

In [9]:
from datetime import datetime
from pyspark.sql import Row
spark=SparkSession(sc)
allTypes=sc.parallelize([Row(i=1,s="string",d=1.0,l=1,
b=True, list=[1,2,3], dict={"s":0}, row=Row(a=1),
time=datetime(2014,8,1,14,1,5))])
df=allTypes.toDF()
df.createOrReplaceTempView("allTypes")
spark.sql('select i+1, d+1, not b, list[1], dict["s"], time, row.a '
           'from allTypes where b and i > 0').collect()

[Row((i + 1)=2, (d + 1)=2.0, (NOT b)=False, list[1]=2, dict[s]=0, time=datetime.datetime(2014, 8, 1, 14, 1, 5), a=1)]

In [10]:

from pyspark.sql.functions import col, count, lit, max
from pyspark.sql import Observation
df=spark.createDataFrame([["Alice", 2], ["Bob", 5]], ["name","age"])
observation=Observation("my metrics")
observed_df=df.observe(observation, count(lit(1)).alias("count"),max(col("age")))
observed_df.count()
2
observation.get
{'count':2, 'max(age)':5}

{'count': 2, 'max(age)': 5}

In [11]:
row=Row(name="Alice", age=11)
row
Row(name='Alice',age=11)
row['name'],row['age']
('Alice',11)
row.name, row.age
('Alice',11)
'name' in row
True
'wrong_key' in row

False

In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

data =[(Vectors.sparse(4,[(0,1.0),(3,-2.0)]),),
       (Vectors.dense([4.0,5.0,0.0,3.0]),),
       (Vectors.dense([6.0,7.0,0.0,8.0]),),
       (Vectors.sparse(4,[(0,1.0),(3,-2.0)]),),]
df=spark.createDataFrame(data,["features"])
r1=Correlation.corr(df,"features").head()
print("Pearson correlation matrix:\n"+str(r1[0]))
r2=Correlation.corr(df,"features","spearman").head()
print("Spearman correlation matrix:\n"+str(r2[0]))

Pearson correlation matrix:
DenseMatrix([[1.        , 0.99413485,        nan, 0.99493668],
             [0.99413485, 1.        ,        nan, 0.97823198],
             [       nan,        nan, 1.        ,        nan],
             [0.99493668, 0.97823198,        nan, 1.        ]])
Spearman correlation matrix:
DenseMatrix([[ 1.,  1., nan,  1.],
             [ 1.,  1., nan,  1.],
             [nan, nan,  1., nan],
             [ 1.,  1., nan,  1.]])


In [13]:
from pyspark.ml.feature import FeatureHasher
dataset=spark.createDataFrame([
    (2.2, True, "1", "foo"),
    (3.3,False,"2","bar"),
    (4.4,False,"3","baz"),
    (5.5,False,"4","foo")
  
],["real","bool","stringNum","string"])

hasher=FeatureHasher(inputCols=["real","bool","stringNum","string"],
                     outputCol="features")
featurized=hasher.transform(dataset)
featurized.show(truncate=False)

+----+-----+---------+------+--------------------------------------------------------+
|real|bool |stringNum|string|features                                                |
+----+-----+---------+------+--------------------------------------------------------+
|2.2 |true |1        |foo   |(262144,[174475,247670,257907,262126],[2.2,1.0,1.0,1.0])|
|3.3 |false|2        |bar   |(262144,[70644,89673,173866,174475],[1.0,1.0,1.0,3.3])  |
|4.4 |false|3        |baz   |(262144,[22406,70644,174475,187923],[1.0,1.0,4.4,1.0])  |
|5.5 |false|4        |foo   |(262144,[70644,101499,174475,257907],[1.0,1.0,5.5,1.0]) |
+----+-----+---------+------+--------------------------------------------------------+



In [14]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

sentenceDataFrame=spark.createDataFrame([
    (0," Hi heared about spark"),
    (1,"I wish Java could use case classes"),
    (2,"Logistic, regression, models,are,meat")

],["id","sentence"])

tokenizer=Tokenizer(inputCol="sentence", outputCol="words")

regexTokenizer=RegexTokenizer(inputCol="sentence", outputCol="words",pattern="\\w")

countTokens=udf(lambda words: len(words), IntegerType())

tokenized=tokenizer.transform(sentenceDataFrame)
tokenized.select("sentence","words")\
    .withColumn("tokens",countTokens(col("words"))).show(truncate=False)

regexTokenized=regexTokenizer.transform(sentenceDataFrame)
regexTokenized.select("sentence","words")\
    .withColumn("tokens",countTokens(col("words"))).show(truncate=False)



+-------------------------------------+------------------------------------------+------+
|sentence                             |words                                     |tokens|
+-------------------------------------+------------------------------------------+------+
| Hi heared about spark               |[, hi, heared, about, spark]              |5     |
|I wish Java could use case classes   |[i, wish, java, could, use, case, classes]|7     |
|Logistic, regression, models,are,meat|[logistic,, regression,, models,are,meat] |3     |
+-------------------------------------+------------------------------------------+------+

+-------------------------------------+------------------+------+
|sentence                             |words             |tokens|
+-------------------------------------+------------------+------+
| Hi heared about spark               |[ ,  ,  ,  ]      |4     |
|I wish Java could use case classes   |[ ,  ,  ,  ,  ,  ]|6     |
|Logistic, regression, models,are,meat|

In [1]:
!git clone https://github.com/apache/spark.git

Cloning into 'spark'...
remote: Enumerating objects: 995403, done.
remote: Counting objects: 100% (622/622), done.
remote: Compressing objects: 100% (365/365), done.
remote: Total 995403 (delta 216), reused 420 (delta 123), pack-reused 994781
Receiving objects: 100% (995403/995403), 456.63 MiB | 18.12 MiB/s, done.
Resolving deltas: 100% (446598/446598), done.
Updating files: 100% (20273/20273), done.


In [23]:
#Stan
from pyspark.ml.feature import StandardScaler
dataFrame=spark.read.format("libsvm").load("/content/spark/data/mllib/sample_libsvm_data.txt")
scaler=StandardScaler(inputCol="features", outputCol="scaledFeatures",
                      withStd=True, withMean=False)
#Compute summary statistics by fitting the standerscaler
scalerModel=scaler.fit(dataFrame)

scaledData=scalerModel.transform(dataFrame)
scaledData.show()

+-----+--------------------+--------------------+
|label|            features|      scaledFeatures|
+-----+--------------------+--------------------+
|  0.0|(692,[127,128,129...|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|(692,[124,125,126...|
